STOCK FUNDAMENTALS - EDA 

In [1]:
# Standard Data Analysis Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [2]:
# Statistical Analysis
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [3]:
# Custom Modules
import sys
sys.path.append('../src')
from data_preprocessing_module import DataPreprocessor
from data_validator import DataValidator


In [67]:
# Visualization Settings
# plt.style.available - run to check available styles
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [6]:
# Configuration
DATA_PATH = Path('../data/raw/20241227_2150')
RANDOM_STATE = 42

In [7]:
# Utility Functions for Financial Calculations
def calculate_missing_percentages(df):
    """Calculate percentage of missing values for each column."""
    missing_percentages = (df.isnull().sum() / len(df)) * 100
    return missing_percentages.sort_values(ascending=False)

In [8]:
def detect_outliers(series, n_std=3):
    """Detect outliers using n standard deviations."""
    mean = np.mean(series)
    std = np.std(series)
    outliers = series[abs(series - mean) > n_std * std]
    return outliers

In [9]:
# Load Data
preprocessor = DataPreprocessor()
validator = DataValidator()    

In [10]:
import os
# from pathlib import Path
# current_working_directory = Path.cwd()
# print(current_working_directory)
print(os.getcwd())
os.chdir('../data/raw/20241227_2150')
print(os.getcwd())
print(os.listdir())


/Users/praveenkumar/Documents/GitHub/stock-classification-valuation/notebooks
/Users/praveenkumar/Documents/GitHub/stock-classification-valuation/data/raw/20241227_2150


In [42]:
data_list = sorted(os.listdir())
print(data_list)


['HDFCBANK_balance_sheet.parquet', 'HDFCBANK_cash_flow.parquet', 'HDFCBANK_historical_1y_1d.parquet', 'HDFCBANK_historical_5y_1wk.parquet', 'HDFCBANK_historical_max_1mo.parquet', 'HDFCBANK_income_statement.parquet', 'HDFCBANK_info.parquet', 'RELIANCE_balance_sheet.parquet', 'RELIANCE_cash_flow.parquet', 'RELIANCE_historical_1y_1d.parquet', 'RELIANCE_historical_5y_1wk.parquet', 'RELIANCE_historical_max_1mo.parquet', 'RELIANCE_income_statement.parquet', 'RELIANCE_info.parquet', 'TCS_balance_sheet.parquet', 'TCS_cash_flow.parquet', 'TCS_historical_1y_1d.parquet', 'TCS_historical_5y_1wk.parquet', 'TCS_historical_max_1mo.parquet', 'TCS_income_statement.parquet', 'TCS_info.parquet']


In [46]:
balance_sheet_list = [i for i in data_list if "balance_sheet" in i]
cash_flow_list = [i for i in data_list if "cash_flow" in i]
income_statement_list = [i for i in data_list if "income_statement" in i]

print("\nBalance Sheet List:",balance_sheet_list)
print("\nCash Flow List:",cash_flow_list)
print("\nIncome statementList:",income_statement_list)


Balance Sheet List: ['HDFCBANK_balance_sheet.parquet', 'RELIANCE_balance_sheet.parquet', 'TCS_balance_sheet.parquet']

Cash Flow List: ['HDFCBANK_cash_flow.parquet', 'RELIANCE_cash_flow.parquet', 'TCS_cash_flow.parquet']

Income statementList: ['HDFCBANK_income_statement.parquet', 'RELIANCE_income_statement.parquet', 'TCS_income_statement.parquet']


In [49]:
companies = list(set([file.split('_')[0] for file in balance_sheet_list + cash_flow_list +income_statement_list]))
companies


['TCS', 'RELIANCE', 'HDFCBANK']

In [52]:
company_data = {company: [] for company in companies}
for file in balance_sheet_list + cash_flow_list + income_statement_list :
    company = file.split('_')[0]
    company_data[company].append(file)
    
# Print results
print("Companies:", companies)
for company in companies:
    print(f"{company}:", company_data[company])

Companies: ['TCS', 'RELIANCE', 'HDFCBANK']
TCS: ['TCS_balance_sheet.parquet', 'TCS_cash_flow.parquet', 'TCS_income_statement.parquet']
RELIANCE: ['RELIANCE_balance_sheet.parquet', 'RELIANCE_cash_flow.parquet', 'RELIANCE_income_statement.parquet']
HDFCBANK: ['HDFCBANK_balance_sheet.parquet', 'HDFCBANK_cash_flow.parquet', 'HDFCBANK_income_statement.parquet']


In [53]:
company_data

{'TCS': ['TCS_balance_sheet.parquet',
  'TCS_cash_flow.parquet',
  'TCS_income_statement.parquet'],
 'RELIANCE': ['RELIANCE_balance_sheet.parquet',
  'RELIANCE_cash_flow.parquet',
  'RELIANCE_income_statement.parquet'],
 'HDFCBANK': ['HDFCBANK_balance_sheet.parquet',
  'HDFCBANK_cash_flow.parquet',
  'HDFCBANK_income_statement.parquet']}

In [71]:
bs_data=balance_sheet_list[2]
print(f"Reading data from {bs_data}")
bs_df=pd.read_parquet(bs_data)
print(f"Shape of {bs_data} : {bs_df.shape}")
bs_df.head()

Reading data from TCS_balance_sheet.parquet
Shape of TCS_balance_sheet.parquet : (79, 5)


,2024-03-31,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Treasury Shares Number,0.000,NaN,0.000,0.000,NaN
Ordinary Shares Number,3618087518.000,3659051373.000,3659051373.000,3699051373.000,NaN
Share Issued,3618087518.000,3659051373.000,3659051373.000,3699051373.000,NaN
Total Debt,80210000000.000,76880000000.000,78180000000.000,77950000000.000,NaN
Tangible Book Value,880870000000.000,876030000000.000,861180000000.000,841550000000.000,NaN


In [73]:
bs_df.index

Index(['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued',
       'Total Debt', 'Tangible Book Value', 'Invested Capital',
       'Working Capital', 'Net Tangible Assets', 'Capital Lease Obligations',
       'Common Stock Equity', 'Total Capitalization',
       'Total Equity Gross Minority Interest', 'Minority Interest',
       'Stockholders Equity', 'Other Equity Interest',
       'Fixed Assets Revaluation Reserve', 'Retained Earnings',
       'Additional Paid In Capital', 'Capital Stock', 'Common Stock',
       'Total Liabilities Net Minority Interest',
       'Total Non Current Liabilities Net Minority Interest',
       'Non Current Pension And Other Postretirement Benefit Plans',
       'Tradeand Other Payables Non Current', 'Non Current Deferred Revenue',
       'Non Current Deferred Taxes Liabilities',
       'Long Term Debt And Capital Lease Obligation',
       'Long Term Capital Lease Obligation', 'Current Liabilities',
       'Other Current Liabilities',
      

In [65]:
bs_df

,2024-03-31,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Treasury Shares Number,0.0,NaN,0.0,0.0,NaN
Ordinary Shares Number,3618087518.0,3659051373.0,3659051373.0,3699051373.0,NaN
Share Issued,3618087518.0,3659051373.0,3659051373.0,3699051373.0,NaN
Total Debt,80210000000.0,76880000000.0,78180000000.0,77950000000.0,NaN
Tangible Book Value,880870000000.0,876030000000.0,861180000000.0,841550000000.0,NaN
Invested Capital,904890000000.0,904240000000.0,891390000000.0,864330000000.0,NaN
Working Capital,668800000000.0,667120000000.0,659590000000.0,651250000000.0,NaN
Net Tangible Assets,880870000000.0,876030000000.0,861180000000.0,841550000000.0,NaN
Capital Lease Obligations,80210000000.0,76880000000.0,78180000000.0,77950000000.0,NaN
Common Stock Equity,904890000000.0,904240000000.0,891390000000.0,864330000000.0,NaN


In [66]:
is_data=income_statement_list[2]
print(f"Reading data from {is_data}")
is_df=pd.read_parquet(is_data)
print(f"Shape of {is_data} : {is_df.shape}")
is_df

Reading data from TCS_income_statement.parquet
Shape of TCS_income_statement.parquet : (47, 4)


,2024-03-31,2023-03-31,2022-03-31,2021-03-31
Tax Effect Of Unusual Items,-1924680000.0,277560000.0,238080000.0,-2821120000.0
Tax Rate For Calcs,0.3,0.3,0.3,0.3
Normalized EBITDA,685060000000.0,626000000000.0,569820000000.0,495640000000.0
Total Unusual Items,-7460000000.0,1080000000.0,930000000.0,-11020000000.0
Total Unusual Items Excluding Goodwill,-7460000000.0,1080000000.0,930000000.0,-11020000000.0
Net Income From Continuing Operation Net Minority Interest,459080000000.0,421470000000.0,383270000000.0,324300000000.0
Reconciled Depreciation,49850000000.0,50220000000.0,46040000000.0,40650000000.0
Reconciled Cost Of Revenue,1328710000000.0,1197590000000.0,1002670000000.0,868750000000.0
EBITDA,677600000000.0,627080000000.0,570750000000.0,484620000000.0
EBIT,627750000000.0,576860000000.0,524710000000.0,443970000000.0


In [68]:
cf_data=cash_flow_list[2]
print(f"Reading data from {cf_data}")
cf_df=pd.read_parquet(cf_data)
print(f"Shape of {cf_data} : {cf_df.shape}")
cf_df

Reading data from TCS_cash_flow.parquet
Shape of TCS_cash_flow.parquet : (51, 5)


,2024-03-31,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Free Cash Flow,416640000000.000,388650000000.000,369540000000.000,356260000000.000,NaN
Repurchase Of Capital Stock,-170000000000.000,0.000,-180000000000.000,-160000000000.000,NaN
Issuance Of Capital Stock,NaN,NaN,1620000000.000,1600000000.000,0.000
Capital Expenditure,-26740000000.000,-31000000000.000,-29950000000.000,-31760000000.000,NaN
End Cash Position,90160000000.000,71230000000.000,124880000000.000,68580000000.000,NaN
Beginning Cash Position,71230000000.000,124880000000.000,68580000000.000,86460000000.000,NaN
Effect Of Exchange Rate Changes,650000000.000,5090000000.000,1590000000.000,1730000000.000,NaN
Changes In Cash,18280000000.000,-58740000000.000,54710000000.000,-19610000000.000,NaN
Financing Cash Flow,-485360000000.000,-478780000000.000,-335810000000.000,-326340000000.000,NaN
Net Other Financing Charges,-39590000000.000,-41740000000.000,-420000000.000,-37260000000.000,NaN
